In [1]:
#pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 KB 11.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp39-cp39-linux_x86_64.whl size=3193684 sha256=ba500be14903a9d792ce7d26a9fd4e6ff2f9c7639cf4d6c8fb38fe529b56ae9d
  Stored in directory: /root/.cache/pip/wheels/c6/3a/46/9b17b3512bdf283c6cb84f59929cdd5199d4e754d596d22784
Successfully built scikit-surprise


In [2]:
import pandas as pd
import numpy as np
import warnings
# 경고 제거
warnings.filterwarnings("ignore")

In [3]:
#from google.colab import files
#files.upload();

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
df = pd.read_csv('/content/drive/MyDrive/ml-100k/u.data', sep = '\t', header = None)

In [6]:
df.head

<bound method NDFrame.head of          0     1  2          3
0      196   242  3  881250949
1      186   302  3  891717742
2       22   377  1  878887116
3      244    51  2  880606923
4      166   346  1  886397596
...    ...   ... ..        ...
99995  880   476  3  880175444
99996  716   204  5  879795543
99997  276  1090  1  874795795
99998   13   225  2  882399156
99999   12   203  3  879959583

[100000 rows x 4 columns]>

In [7]:
df.columns = ['user_id', 'item_id', 'rating', 'timestamp']

In [8]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [9]:
df['timestamp']

0        881250949
1        891717742
2        878887116
3        880606923
4        886397596
           ...    
99995    880175444
99996    879795543
99997    874795795
99998    882399156
99999    879959583
Name: timestamp, Length: 100000, dtype: int64

In [10]:
from datetime import datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='s')

In [11]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3,1997-12-04 15:55:49
1,186,302,3,1998-04-04 19:22:22
2,22,377,1,1997-11-07 07:18:36
3,244,51,2,1997-11-27 05:02:03
4,166,346,1,1998-02-02 05:33:16


In [12]:
len(df.user_id.unique()) # 943명의 유저, item은 1,682개, 평가는 10만개)

943

In [13]:
df.rating.unique()

array([3, 1, 2, 4, 5])

In [14]:
df.isnull().sum()

user_id      0
item_id      0
rating       0
timestamp    0
dtype: int64

In [15]:
df.duplicated().sum()

0

# Baseline model : 최빈값
- 모든 유저가 가장 많이 경험한 영화를 추천

In [62]:
df.groupby('item_id').count().sort_values(by='user_id', ascending=False)

,user_id,rating,timestamp
item_id,,,
50,583,583,583
258,509,509,509
100,508,508,508
181,507,507,507
294,485,485,485
...,...,...,...
1576,1,1,1
1577,1,1,1
1348,1,1,1


# 1. CB(Content Based recommendation)
- u.item, u.user : 영화정보, 유저정보 데이터 관련

In [63]:
import chardet

with open('/content/drive/MyDrive/ml-100k/u.item', 'rb') as f:
  enc = chardet.detect(f.read())

KeyboardInterrupt: ignored

In [ ]:
enc

In [ ]:
# chardet 안쓰면 인코더(utf-8)로 안 읽힘 - chardet: 읽어보고 가능한 인코딩 알아주는 기능
df_item = pd.read_csv('/content/drive/MyDrive/ml-100k/u.item', header = None, encoding = enc['encoding'], sep = '|')
df_item.head()

In [ ]:
item_columns = ['movie_id', 'movie_title', 'release_date',  'video_release_date', 'IMDb_URL', 'unknown', 'Action', 'Adventure', 'Animation', "Children's", "Comedy", "Crime", "Documentary",
                "Drama", 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']

In [ ]:
df_item.columns = item_columns

In [ ]:
df_item[df_item['release_date'].isnull()]

In [ ]:
df_item = df_item.drop(columns = ['video_release_date', 'IMDb_URL'])

In [ ]:
df_item = df_item.drop(axis = 0, index =266)

In [ ]:
df_item['movie_title'] = df_item['movie_title'].str[:-7]

In [ ]:
df_item['release_date'].str[3:6].unique()

In [ ]:
df_item.loc[1372,'release_date'] = '04-Feb-1971'

In [ ]:
df_item.loc[1372,'release_date']

In [ ]:
df_item['release_date'].str[3:6].unique()

In [ ]:
df_item['release_date'].str[:2].unique()

In [ ]:
df_item['release_date'].str[-4:].unique()

In [ ]:
month = {
    'Jan' : '01',
    'Feb' : '02',
    'Mar' : '03',
    'Apr' : '04',
    'May' : '05',
    'Jun' : '06',
    'Jul' : '07',
    'Aug' : '08',
    'Sep' : '09',
    'Oct' : '10',
    'Nov' : '11',
    'Dec' : '12'
}

In [ ]:
month_numed = pd.Series([month[x] for x in df_item['release_date'].str[3: 6]])

In [ ]:
len(df_item)

In [ ]:
len(month_numed)

In [ ]:
df_item['release_date'].tail()

In [ ]:
df_item.reset_index(inplace=True)

In [ ]:
month_numed.tail()

In [ ]:
month_numed.unique()

In [ ]:
df_item['release_date'] = df_item['release_date'].str[-4:]+month_numed+df_item['release_date'].str[:2]

In [ ]:
df_item['release_date'] = pd.to_datetime(df_item['release_date'], format = '%Y%m%d')

In [ ]:
df_item.tail()

In [ ]:
df.describe()

In [ ]:
df_item[['unknown', 'Action',
       'Adventure', 'Animation', "Children's", 'Comedy', 'Crime',
       'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical',
       'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']].describe()

In [ ]:
df_user = pd.read_csv('/content/drive/MyDrive/ml-100k/u.user', header = None, sep = "|")

In [ ]:
df_user.head()

In [ ]:
usercol = ['user_id','age','gender','occupation','zip_code']
df_user.columns = usercol
df_user.head()

In [ ]:
df_user.isnull().sum()

In [ ]:
df_user.duplicated().sum()

In [ ]:
len(df.user_id.unique())

In [ ]:
len(df_user.user_id.unique())

In [ ]:
df_user.gender.unique()

In [ ]:
df_user.occupation.unique()

# 2. CF(Collaborative filtering) : 협업 필터링
  1) User-based CF (메모리 기반 - 유저기반 CF)

In [65]:
# "유저별 영화 평점"을 알아보기 위해 pivot table 활용, 결측치 처리
user_item_rating = df.pivot_table('rating', index = 'user_id', columns = 'item_id')
user_item_rating.fillna(0, inplace=True)
user_item_rating.head()

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
user_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,3.0,4.0,3.0,3.0,5.0,4.0,1.0,5.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [66]:
from sklearn.metrics.pairwise import cosine_similarity

# 유저들의 평점을 행(관측값)으로 갖는 데이터프레임의 코사인유사도를 계산
user_based_cs = cosine_similarity(user_item_rating)
user_based_cs

array([[1.        , 0.16693098, 0.04745954, ..., 0.14861694, 0.17950788,
        0.39817474],
       [0.16693098, 1.        , 0.11059132, ..., 0.16148478, 0.17226781,
        0.10579788],
       [0.04745954, 0.11059132, 1.        , ..., 0.10124256, 0.13341615,
        0.02655587],
       ...,
       [0.14861694, 0.16148478, 0.10124256, ..., 1.        , 0.1016418 ,
        0.09511958],
       [0.17950788, 0.17226781, 0.13341615, ..., 0.1016418 , 1.        ,
        0.18246466],
       [0.39817474, 0.10579788, 0.02655587, ..., 0.09511958, 0.18246466,
        1.        ]])

In [67]:
user_based_cs.shape # 유저기반cf(코사인유사도) 결과는 유저들간의 평점 유사도이다.

(943, 943)

In [68]:
#데이터프레임화
user_based_cf = pd.DataFrame(data = user_based_cs, index = user_item_rating.index, columns = user_item_rating.index)
user_based_cf

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
user_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.166931,0.047460,0.064358,0.378475,0.430239,0.440367,0.319072,0.078138,0.376544,...,0.369527,0.119482,0.274876,0.189705,0.197326,0.118095,0.314072,0.148617,0.179508,0.398175
2,0.166931,1.000000,0.110591,0.178121,0.072979,0.245843,0.107328,0.103344,0.161048,0.159862,...,0.156986,0.307942,0.358789,0.424046,0.319889,0.228583,0.226790,0.161485,0.172268,0.105798
3,0.047460,0.110591,1.000000,0.344151,0.021245,0.072415,0.066137,0.083060,0.061040,0.065151,...,0.031875,0.042753,0.163829,0.069038,0.124245,0.026271,0.161890,0.101243,0.133416,0.026556
4,0.064358,0.178121,0.344151,1.000000,0.031804,0.068044,0.091230,0.188060,0.101284,0.060859,...,0.052107,0.036784,0.133115,0.193471,0.146058,0.030138,0.196858,0.152041,0.170086,0.058752
5,0.378475,0.072979,0.021245,0.031804,1.000000,0.237286,0.373600,0.248930,0.056847,0.201427,...,0.338794,0.080580,0.094924,0.079779,0.148607,0.071459,0.239955,0.139595,0.152497,0.313941
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
939,0.118095,0.228583,0.026271,0.030138,0.071459,0.111852,0.107027,0.095898,0.039852,0.071460,...,0.066039,0.431154,0.258021,0.226449,0.432666,1.000000,0.087687,0.180029,0.043264,0.144250
940,0.314072,0.226790,0.161890,0.196858,0.239955,0.352449,0.329925,0.246883,0.120495,0.342961,...,0.327153,0.107024,0.187536,0.181317,0.175158,0.087687,1.000000,0.145152,0.261376,0.241028
941,0.148617,0.161485,0.101243,0.152041,0.139595,0.144446,0.059993,0.146145,0.143245,0.090305,...,0.046952,0.203301,0.288318,0.234211,0.313400,0.180029,0.145152,1.000000,0.101642,0.095120


In [69]:
# 어떤 유저 아이디를 알 때, 유사한 평점을 준 유저를 5명 추천한다. (User-based)
def get_user_id(id):
  return user_based_cf[id].sort_values(ascending=False)[:5]

In [70]:
get_user_id(1)

user_id
1      1.000000
916    0.569066
864    0.547548
268    0.542077
92     0.540534
Name: 1, dtype: float64

# 2) Item-based CF : 아이템 기반 CF



In [71]:
# "아이템별 평점"을 알아보기 위해 pivot table 활용, 결측치 처리
item_user_rating = df.pivot_table('rating', index = 'item_id', columns = 'user_id')
item_user_rating.fillna(0, inplace=True)
item_user_rating.head()

user_id,1,2,3,4,5,6,7,8,9,10,...,934,935,936,937,938,939,940,941,942,943
item_id,,,,,,,,,,,,,,,,,,,,,
1,5.0,4.0,0.0,0.0,4.0,4.0,0.0,0.0,0.0,4.0,...,2.0,3.0,4.0,0.0,4.0,0.0,0.0,5.0,0.0,0.0
2,3.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,...,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
3,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,3.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,4.0,...,5.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0
5,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [75]:
# item의 평점을 행(관측값)으로 갖는 데이터프레임의 코사인유사도를 계산
item_based_cs = cosine_similarity(item_user_rating)
item_based_cs

array([[1.        , 0.40238218, 0.33024479, ..., 0.        , 0.04718307,
        0.04718307],
       [0.40238218, 1.        , 0.27306918, ..., 0.        , 0.07829936,
        0.07829936],
       [0.33024479, 0.27306918, 1.        , ..., 0.        , 0.        ,
        0.09687505],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.        ,
        0.        ],
       [0.04718307, 0.07829936, 0.        , ..., 0.        , 1.        ,
        0.        ],
       [0.04718307, 0.07829936, 0.09687505, ..., 0.        , 0.        ,
        1.        ]])

In [76]:
item_based_cs.shape # item기반cf(코사인유사도) 결과는 item간의 평점 유사도이다.

(1682, 1682)

In [77]:
#데이터프레임화
item_based_cf = pd.DataFrame(data = item_based_cs, index = item_user_rating.index, columns = item_user_rating.index)
item_based_cf

item_id,1,2,3,4,5,6,7,8,9,10,...,1673,1674,1675,1676,1677,1678,1679,1680,1681,1682
item_id,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.402382,0.330245,0.454938,0.286714,0.116344,0.620979,0.481114,0.496288,0.273935,...,0.035387,0.0,0.000000,0.000000,0.035387,0.0,0.0,0.0,0.047183,0.047183
2,0.402382,1.000000,0.273069,0.502571,0.318836,0.083563,0.383403,0.337002,0.255252,0.171082,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.078299,0.078299
3,0.330245,0.273069,1.000000,0.324866,0.212957,0.106722,0.372921,0.200794,0.273669,0.158104,...,0.000000,0.0,0.000000,0.000000,0.032292,0.0,0.0,0.0,0.000000,0.096875
4,0.454938,0.502571,0.324866,1.000000,0.334239,0.090308,0.489283,0.490236,0.419044,0.252561,...,0.000000,0.0,0.094022,0.094022,0.037609,0.0,0.0,0.0,0.056413,0.075218
5,0.286714,0.318836,0.212957,0.334239,1.000000,0.037299,0.334769,0.259161,0.272448,0.055453,...,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.094211
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1678,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1679,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000
1680,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.000000,0.000000,0.000000,1.0,1.0,1.0,0.000000,0.000000


In [78]:
# 어떤 item 아이디를 알 때, 유사한 item 5개 추천한다. (item-based)
def get_item_id(id):
  return item_based_cf[id].sort_values(ascending=False)[:5]

In [81]:
get_item_id(1)

item_id
1      1.000000
50     0.734572
181    0.699925
121    0.689786
117    0.664555
Name: 1, dtype: float64

# 협업필터링(Collaborative Filtering) surprise라이브러리로 구현하기
- 협업필터링은 일반적인 memory(user, item) based 그리고 머신러닝을 이용해서 예측을 하는 model based의 필터링, 그리고 거기에 더해 희소행렬을 아예 임베딩등을 통해 차원을 변경해 여러 레이어로 구성된 딥러닝 필터링까지도 가능하다. 여기서는, surprise라이브러리로 구현이 가능함을 보인다.


(http://machinelearningkorea.com/2019/05/18/surprise-%EB%9D%BC%EC%9D%B4%EB%B8%8C%EB%9F%AC%EB%A6%AC%EB%A5%BC-%EC%9D%B4%EC%9A%A9%ED%95%9C-%ED%98%91%EC%97%85%ED%95%84%ED%84%B0%EB%A7%81collaborative-filtering-%ED%8C%8C%EC%9D%B4%EC%8D%AC-%EC%B9%98/)

In [79]:
import surprise
from surprise import Reader, Dataset

In [80]:
# Scale만 지정해주고, 필요한 userId, movieId, rating에 대한 컬럼만 설정
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(df[['user_id', 'item_id', 'rating']], reader)